### How much such a not-for-profit organization pay it's top management?  Not-for-profit organizations devoted to specific causes have a huge amount of variablity (revenue, number of volunteers, ect.).  How much should not-for-profits award their top managment?
### Data source:

The data for this project was aquired from Open990(https://www.open990.org/catalog/), an organization that aggregates and provides not-for-profit tax return data made public by the IRS.  This dataset has great documentation about what the attributes mean, however this only includes data for 1 year(2016).  The organization that provides this data also provides analytics services with this data for a fee. (https://appliednonprofitresearch.com/customdata/)

overview of data provided by open990
https://medium.com/@open990/the-irs-990-e-file-dataset-getting-to-the-chocolatey-center-of-data-deliciousness-90f66097a600

NTEE Descriptions have a huge amount of specificity https://learn.guidestar.org/help/ntee-codes
Based on NTEE categories, what salary should be paid for top positions?

### Research questions to consider:

1. What should a not-for-profit be paying it's top management?  
1. By using a model to establish this amount I can also identify organizations who are likely paying their management more than they should.



### Other things to consider:
Organizations with segregation of duties concerns "Clerk, Treas., Dir."


### Process:
1. Use governance characteristics to cluster orgs.
1. Use cluster information from above to then predict Trustee or director salary

In [28]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [38]:
df_original = pd.read_csv('data_sets/Compensation_charities_Open990.csv', 
                 header=1, nrows=5)

In [41]:
df_original.head(1)

,EIN,Period,Person ID,Title,Hours,Hours related org,Trustee or director,Institutional trustee,Officer,Key employee,Highest compensated,Former,Reportable comp,Reportable comp related org,Other comp all,Health and benefits,Other comp,Base,Base related org,Bonus,Bonus related org,Other,Other related org,Retire,Retire related org,Nontax benefit,Nontax benefit related org,Sum comp,Sum comp related org,Prior deferred,Prior deferred related org,Org name,Subsection,NTEE code,NTEE description,Street,City,State,ZIP,Formation yr,Phone,Website,Assets,Liabilities,Expenses,Revenue,Form type,Efile ID
0,10877614,201712,y7drw7umAm8SK9fWY7aFCA,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006,3174960916,WWW.HEARTCHANGEINDY.ORG,2112293,92283,500749,1792933,990,201803199349308805


In [61]:
# import columns that interest me
cols = ['EIN',
       'Period',
       'Title',
        'Hours',
       'Trustee or director',
       'Reportable comp',
       'Sum comp',
        'Org name',
        'Subsection',
        'NTEE code',
        'NTEE description',
       'State',
       'Formation yr',
       'Assets',
        'Liabilities',
        'Expenses',
        'Revenue']

df = pd.read_csv('data_sets/Compensation_charities_Open990.csv', 
                 header=1,
                usecols=cols)

In [62]:
df.shape

(3990749, 17)

In [74]:
# convert column labels to lower case and remove spaces from column names
df.columns = df.columns.str.replace(' ', '_')
df.columns = map(str.lower, df.columns)

In [76]:
df.head()

,ein,period,title,hours,trustee_or_director,reportable_comp,sum_comp,org_name,subsection,ntee_code,ntee_description,state,formation_yr,assets,liabilities,expenses,revenue
0,10877614,201712,BOARD MEMBER,1.0,True,0.0,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",IN,2006.0,2112293,92283.0,500749.0,1792933.0
1,10877614,201712,BOARD MEMBER,1.0,True,0.0,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",IN,2006.0,2112293,92283.0,500749.0,1792933.0
2,10877614,201712,BOARD MEMBER,1.0,True,0.0,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",IN,2006.0,2112293,92283.0,500749.0,1792933.0
3,10877614,201712,BOARD MEMBER,1.0,True,0.0,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",IN,2006.0,2112293,92283.0,500749.0,1792933.0
4,10877614,201712,BOARD MEMBER,1.0,True,0.0,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",IN,2006.0,2112293,92283.0,500749.0,1792933.0


In [25]:
# how many employees relate to 2017
df[df.Period.apply(lambda x: str(x)[0:4]) == '2017'].shape

(1113445, 17)

In [30]:
df[(df['Trustee or director'] == True) &
  (df['Reportable comp'] != 0) &
   (df['Reportable comp'] != np.nan) &
   (df.Period.apply(lambda x: str(x)[0:4]) == '2017')
  ].shape

(44302, 17)

In [79]:
df[(df.trustee_or_director == True) &
  (df.reportable_comp != 0) &
   (df.reportable_comp != np.nan) &
   (df.period.apply(lambda x: str(x)[0:4]) == '2017')
  ].head()

,ein,period,title,hours,trustee_or_director,reportable_comp,sum_comp,org_name,subsection,ntee_code,ntee_description,state,formation_yr,assets,liabilities,expenses,revenue
6,10877614,201712,BOARD MEMBER/MANAGING D,65.0,True,55385.0,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",IN,2006.0,2112293,92283.0,500749.0,1792933.0
7,10877614,201712,BOARD MEMBER,40.0,True,60000.0,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",IN,2006.0,2112293,92283.0,500749.0,1792933.0
154,10884383,201712,PRESIDENT,20.0,True,71062.0,NaN,MILAGRE KIDS SCHOOLS INC,501(c)(3),B28,Specialized Education Institutions,PA,2007.0,97123,13093.0,1251188.0,1294636.0
158,10884383,201712,VICE PRESIDENT,10.0,True,9010.0,NaN,MILAGRE KIDS SCHOOLS INC,501(c)(3),B28,Specialized Education Institutions,PA,2007.0,97123,13093.0,1251188.0,1294636.0
459,10906593,201712,VP OF FINANC,17.0,True,87452.0,NaN,MANAGEMENT SERVICES INC,501(c)(3),S02,Management and technical assistance - Communit...,CA,2008.0,4161179,120847.0,881055.0,795668.0


In [69]:
df.head()

,EIN,Period,Person ID,Title,Hours,Hours related org,Trustee or director,Institutional trustee,Officer,Key employee,Highest compensated,Former,Reportable comp,Reportable comp related org,Other comp all,Health and benefits,Other comp,Base,Base related org,Bonus,Bonus related org,Other,Other related org,Retire,Retire related org,Nontax benefit,Nontax benefit related org,Sum comp,Sum comp related org,Prior deferred,Prior deferred related org,Org name,Subsection,NTEE code,NTEE description,Street,City,State,ZIP,Formation yr,Phone,Website,Assets,Liabilities,Expenses,Revenue,Form type,Efile ID
0,10877614,201712,y7drw7umAm8SK9fWY7aFCA,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
1,10877614,201712,NPcAAprw13Optnp4Bio5og,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
2,10877614,201712,AcFe9lOktfzX4yAlRDDHsA,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
3,10877614,201712,MzSmGy0aHY9k7hkE_Y3EBQ,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
4,10877614,201712,hhCYV22sDcQfPzrFejcf-g,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805


In [64]:
# because of the high variance to each of these categories
# cluster and label based on characteristics
# to create a more effective organization than 
# just the NTEE code
df.groupby('NTEE code')['Revenue'].agg(['min', 'max', 'var'])

,min,max,var
NTEE code,,,
A01,0.0,7.365021e+07,1.469259e+14
A02,0.0,1.400675e+07,4.696663e+12
A03,-1378019.0,1.427941e+08,2.838741e+14
A05,0.0,1.420806e+07,1.005822e+13
A11,-1463762.0,7.252340e+07,6.811448e+13
...,...,...,...
Y43,-10935589.0,4.359651e+09,6.068262e+16
Y44,10.0,3.904413e+09,3.294273e+17
Y50,-100219.0,1.543805e+08,5.602415e+13


In [61]:
df['Period'].value_counts().sort_index()

201701        643
201702        862
201703       3614
201704       1979
201705       4760
201706      81602
201707       4402
201708       8792
201709      16800
201710       2684
201711       3320
201712     983987
201801       5711
201802       6952
201803      45931
201804      25185
201805      64261
201806    1018386
201807      52774
201808     111790
201809     239001
201810      32875
201811      11487
201812    1077078
201901       6357
201902       7038
201903      37386
201904      16561
201905      22745
201906      90419
201907       2908
201908       1919
201909        515
210805         25
Name: Period, dtype: int64

In [49]:
df['NTEE code'].value_counts()

S41    132385
B11    114217
J40     78004
S80     69231
S20     62799
        ...  
L33        10
P23         9
V31         8
B3O         3
V34         3
Name: NTEE code, Length: 633, dtype: int64

In [50]:
df['NTEE code'].nunique()

633

In [66]:
df['Subsection'].value_counts()

501(c)(3)     3040790
501(c)(6)      396563
501(c)(4)      144899
501(c)(5)      127732
501(c)(7)      101586
501(c)(8)       40372
501(c)(19)      31390
501(c)(9)       26111
501(c)(12)      24519
501(c)(14)      17317
501(c)(2)       14124
501(c)(13)      13432
501(c)(10)       8855
501(c)(25)       1317
501(c)(15)        673
4947(a)(1)        229
501(c)(17)        214
501(c)(29)        144
501(c)(1)          88
501(c)(27)         87
501(c)(16)         85
501(c)(23)         63
501(c)(26)         61
501(c)(11)         39
501(c)(50)         34
501(c)(71)         11
501(c)(18)          8
501(c)(20)          6
Name: Subsection, dtype: int64

In [54]:
df[df['Trustee or director'] == True].shape

(2501613, 48)

In [55]:
df[df['Trustee or director'] == True].head()

,EIN,Period,Person ID,Title,Hours,Hours related org,Trustee or director,Institutional trustee,Officer,Key employee,Highest compensated,Former,Reportable comp,Reportable comp related org,Other comp all,Health and benefits,Other comp,Base,Base related org,Bonus,Bonus related org,Other,Other related org,Retire,Retire related org,Nontax benefit,Nontax benefit related org,Sum comp,Sum comp related org,Prior deferred,Prior deferred related org,Org name,Subsection,NTEE code,NTEE description,Street,City,State,ZIP,Formation yr,Phone,Website,Assets,Liabilities,Expenses,Revenue,Form type,Efile ID
0,10877614,201712,y7drw7umAm8SK9fWY7aFCA,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
1,10877614,201712,NPcAAprw13Optnp4Bio5og,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
2,10877614,201712,AcFe9lOktfzX4yAlRDDHsA,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
3,10877614,201712,MzSmGy0aHY9k7hkE_Y3EBQ,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
4,10877614,201712,hhCYV22sDcQfPzrFejcf-g,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805


In [56]:
df[df['Highest compensated'] == True].shape

(97095, 48)

In [68]:
df[df['Key employee'] == True].shape

(46709, 48)

To handle multiple years, and multiple employees receiving different amounts
    groupby each org, find the mean hours, mean reportable compensation, mean trustee or director salary, institutional trustee, ect.

In [58]:
df[df['Reportable comp'] == 0].shape

(3542467, 48)

In [59]:
df[df['Reportable comp'] != 0].shape

(448282, 48)

In [38]:
df.groupby('Org name')['Org name'].count().sort_values(ascending=False)

Org name
HACKENSACK MERIDIAN HEALTH INC-SUBORDINATES    265
EASTERN MAINE HEALTHCARE SYSTEMS                82
Nationwide Children's Hospital Group Return     76
COUNCIL OF INDEPENDENT COLLEGES                 53
ALLIANCE                                        53
                                              ... 
VIRGINIA GREYHOUND ADOPTION                      1
SOUTH CAROLINA SEAFOOD ALLIANCE                  1
CULTURALLY YOURS INTL                            1
SOUTH FLORIDA DAY OF CARING INC                  1
TRAN QUOC BUDDHIST TEMPLE                        1
Name: Org name, Length: 3178, dtype: int64

In [ ]:
HACKENSACK MERIDIAN HEALTH INC-SUBORDINATES

In [44]:
df[(df['Org name'] == 'HACKENSACK MERIDIAN HEALTH INC-SUBORDINATES')].shape

(265, 48)

In [45]:
df[(df['Org name'] == 'HACKENSACK MERIDIAN HEALTH INC-SUBORDINATES') &
  (df['Trustee or director'] == True)].head()

,EIN,Period,Person ID,Title,Hours,Hours related org,Trustee or director,Institutional trustee,Officer,Key employee,Highest compensated,Former,Reportable comp,Reportable comp related org,Other comp all,Health and benefits,Other comp,Base,Base related org,Bonus,Bonus related org,Other,Other related org,Retire,Retire related org,Nontax benefit,Nontax benefit related org,Sum comp,Sum comp related org,Prior deferred,Prior deferred related org,Org name,Subsection,NTEE code,NTEE description,Street,City,State,ZIP,Formation yr,Phone,Website,Assets,Liabilities,Expenses,Revenue,Form type,Efile ID
8199,10649794,201712,nN1M-BylrPv-8qANemZrgg,Trustee,3.0,0.0,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HACKENSACK MERIDIAN HEALTH INC-SUBORDINATES,501(c)(3),E22,"Hospital, General",C/O TAX DEPT 1350 CAMPUS PARKWAY,NEPTUNE,NJ,07753,NaN,7.327513e+09,WWW.HACKENSACKMERIDIANHEALTH.ORG,3503673072,1.439388e+09,2.380716e+09,2.567369e+09,990,201813189349313906
8200,10649794,201712,SBs4U5i59CJj7Gc3DseqzA,Trustee - PRESIDENT SOMC,55.0,0.0,True,NaN,True,NaN,NaN,NaN,403080.0,0.0,94871.0,NaN,NaN,324611.0,0.0,62295.0,0.0,16174.0,0.0,66454.0,0.0,28417.0,0.0,497951.0,0.0,0.0,0.0,HACKENSACK MERIDIAN HEALTH INC-SUBORDINATES,501(c)(3),E22,"Hospital, General",C/O TAX DEPT 1350 CAMPUS PARKWAY,NEPTUNE,NJ,07753,NaN,7.327513e+09,WWW.HACKENSACKMERIDIANHEALTH.ORG,3503673072,1.439388e+09,2.380716e+09,2.567369e+09,990,201813189349313906
8202,10649794,201712,F2GreyncEcu_5-5fK-wFIA,Trustee,3.0,0.0,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HACKENSACK MERIDIAN HEALTH INC-SUBORDINATES,501(c)(3),E22,"Hospital, General",C/O TAX DEPT 1350 CAMPUS PARKWAY,NEPTUNE,NJ,07753,NaN,7.327513e+09,WWW.HACKENSACKMERIDIANHEALTH.ORG,3503673072,1.439388e+09,2.380716e+09,2.567369e+09,990,201813189349313906
8203,10649794,201712,dsRn45tk2qCtM037Lg2GiA,Trustee,3.0,0.0,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HACKENSACK MERIDIAN HEALTH INC-SUBORDINATES,501(c)(3),E22,"Hospital, General",C/O TAX DEPT 1350 CAMPUS PARKWAY,NEPTUNE,NJ,07753,NaN,7.327513e+09,WWW.HACKENSACKMERIDIANHEALTH.ORG,3503673072,1.439388e+09,2.380716e+09,2.567369e+09,990,201813189349313906
8204,10649794,201712,SSl5mcL5vOXDei5es5R5TQ,Trustee,3.0,0.0,True,NaN,NaN,NaN,NaN,NaN,0.0,27211.0,52279.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HACKENSACK MERIDIAN HEALTH INC-SUBORDINATES,501(c)(3),E22,"Hospital, General",C/O TAX DEPT 1350 CAMPUS PARKWAY,NEPTUNE,NJ,07753,NaN,7.327513e+09,WWW.HACKENSACKMERIDIANHEALTH.ORG,3503673072,1.439388e+09,2.380716e+09,2.567369e+09,990,201813189349313906


In [17]:
df_foundations = pd.read_csv('Compensation_foundations_Open990.csv', nrows=10000, header=1)

In [24]:
df_foundations.head()

,EIN,Period,Person ID,Title,Hours,Compensation,Benefits,Allowances,Org name,Subsection,NTEE code,NTEE description,Street,City,State,ZIP,Phone,Website inspect,Assets,Liabilities,Expenses and disbursements,Revenue PF,Form type,Efile ID
0,10936787,201812,OjrhLMXsWwJQ7DJjFnmedw,TREASURER,3.0,0,NaN,NaN,NARROW GATE FOUNDATION,501(c)(3),T22,Private Independent Foundations,98 NUTTALL ROAD,RIVERSIDE,IL,60546,6.305464e+09,NaN,147346,0,71839,60933,990PF,201941799349101009
1,10936787,201812,PfxAhwGU2Nafixszy1QA4Q,DIRECTOR,3.0,0,NaN,NaN,NARROW GATE FOUNDATION,501(c)(3),T22,Private Independent Foundations,98 NUTTALL ROAD,RIVERSIDE,IL,60546,6.305464e+09,NaN,147346,0,71839,60933,990PF,201941799349101009
2,10936787,201812,OhLS2aOqUl2w5PnTvmPI_Q,PRES/SEC,10.0,0,NaN,NaN,NARROW GATE FOUNDATION,501(c)(3),T22,Private Independent Foundations,98 NUTTALL ROAD,RIVERSIDE,IL,60546,6.305464e+09,NaN,147346,0,71839,60933,990PF,201941799349101009
3,10939828,201812,RJwFEd8bXfon5Zui7xRlmg,VICE PRESIDENT,1.0,0,0.0,0.0,Agee Family Foundation,501(c)(3),T20,Private Grantmaking Foundations,1310 W SAM HOUSTON PARKWAY N,HOUSTON,TX,77043,7.134632e+09,NaN,9367821,0,465812,379254,990PF,201941839349100404
4,10939828,201812,2ZHL1IgQT-mWPhBIWnKSlw,PRESIDENT,1.0,0,0.0,0.0,Agee Family Foundation,501(c)(3),T20,Private Grantmaking Foundations,1310 W SAM HOUSTON PARKWAY N,HOUSTON,TX,77043,7.134632e+09,NaN,9367821,0,465812,379254,990PF,201941839349100404


In [30]:
df_foundations[df_foundations['Org name'] == 'NARROW GATE FOUNDATION']

,EIN,Period,Person ID,Title,Hours,Compensation,Benefits,Allowances,Org name,Subsection,NTEE code,NTEE description,Street,City,State,ZIP,Phone,Website inspect,Assets,Liabilities,Expenses and disbursements,Revenue PF,Form type,Efile ID
0,10936787,201812,OjrhLMXsWwJQ7DJjFnmedw,TREASURER,3.0,0,NaN,NaN,NARROW GATE FOUNDATION,501(c)(3),T22,Private Independent Foundations,98 NUTTALL ROAD,RIVERSIDE,IL,60546,6.305464e+09,NaN,147346,0,71839,60933,990PF,201941799349101009
1,10936787,201812,PfxAhwGU2Nafixszy1QA4Q,DIRECTOR,3.0,0,NaN,NaN,NARROW GATE FOUNDATION,501(c)(3),T22,Private Independent Foundations,98 NUTTALL ROAD,RIVERSIDE,IL,60546,6.305464e+09,NaN,147346,0,71839,60933,990PF,201941799349101009
2,10936787,201812,OhLS2aOqUl2w5PnTvmPI_Q,PRES/SEC,10.0,0,NaN,NaN,NARROW GATE FOUNDATION,501(c)(3),T22,Private Independent Foundations,98 NUTTALL ROAD,RIVERSIDE,IL,60546,6.305464e+09,NaN,147346,0,71839,60933,990PF,201941799349101009


In [29]:
df_foundations['NTEE code'].nunique()

244

In [28]:
df_foundations['NTEE code'].value_counts().head(30)

T20    2574
T22    1834
T90     542
B82     406
L99     228
B11     185
A82     168
P75     159
T21     130
A50     127
E11     113
L22     111
A80     104
S20      83
X20      82
T03      79
T23      75
T99      71
L80      63
P99      61
B20      57
S41      50
S80      50
A99      49
D20      47
E99      47
B70      45
N12      44
S21      43
R24      42
Name: NTEE code, dtype: int64

In [19]:
df_foundations['NTEE description'].value_counts()

Private Grantmaking Foundations                         2574
Private Independent Foundations                         1834
Named Trusts/Foundations N.E.C.                          542
Scholarships, Student Financial Aid Services, Awards     406
Housing, Shelter N.E.C.                                  228
                                                        ... 
Rehabilitative Medical Services                            1
Alliance/advocacy organization - Housing                   1
Food Banks, Food Pantries                                  1
Employment Procurement Assistance, Job Training            1
Developmentally Disabled Centers, Services                 1
Name: NTEE description, Length: 229, dtype: int64

In [21]:
df_foundations['Title'].value_counts()

TRUSTEE                2762
DIRECTOR               1677
Trustee                 576
Director                545
PRESIDENT               515
                       ... 
Clerk, Treas., Dir.       1
Scholarship Officer       1
SUCCESSOR TRT             1
MEMBERSHIP CHAIR          1
Pres./Vice-Chair          1
Name: Title, Length: 1052, dtype: int64

In [9]:
df.head()

,EIN,Period,Person ID,Title,Hours,Hours related org,Trustee or director,Institutional trustee,Officer,Key employee,Highest compensated,Former,Reportable comp,Reportable comp related org,Other comp all,Health and benefits,Other comp,Base,Base related org,Bonus,Bonus related org,Other,Other related org,Retire,Retire related org,Nontax benefit,Nontax benefit related org,Sum comp,Sum comp related org,Prior deferred,Prior deferred related org,Org name,Subsection,NTEE code,NTEE description,Street,City,State,ZIP,Formation yr,Phone,Website,Assets,Liabilities,Expenses,Revenue,Form type,Efile ID
0,10877614,201712,y7drw7umAm8SK9fWY7aFCA,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
1,10877614,201712,NPcAAprw13Optnp4Bio5og,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
2,10877614,201712,AcFe9lOktfzX4yAlRDDHsA,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
3,10877614,201712,MzSmGy0aHY9k7hkE_Y3EBQ,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
4,10877614,201712,hhCYV22sDcQfPzrFejcf-g,BOARD MEMBER,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805


In [20]:
df[df['Reportable comp'] > 0].head()

,EIN,Period,Person ID,Title,Hours,Hours related org,Trustee or director,Institutional trustee,Officer,Key employee,Highest compensated,Former,Reportable comp,Reportable comp related org,Other comp all,Health and benefits,Other comp,Base,Base related org,Bonus,Bonus related org,Other,Other related org,Retire,Retire related org,Nontax benefit,Nontax benefit related org,Sum comp,Sum comp related org,Prior deferred,Prior deferred related org,Org name,Subsection,NTEE code,NTEE description,Street,City,State,ZIP,Formation yr,Phone,Website,Assets,Liabilities,Expenses,Revenue,Form type,Efile ID
6,10877614,201712,7I-r1Fu7Vw0W_sYXaIWNNQ,BOARD MEMBER/MANAGING D,65.0,NaN,True,NaN,NaN,NaN,NaN,NaN,55385,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
7,10877614,201712,FsnoEHXJ9ieEdcBs-ew29w,BOARD MEMBER,40.0,NaN,True,NaN,NaN,NaN,NaN,NaN,60000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEART CHANGE MINISTRIES INC,501(c)(3),G43,"Heart and Circulatory System Diseases, Disorders",4201 E 16TH ST,INDIANAPOLIS,IN,46201,2006.0,3.174961e+09,WWW.HEARTCHANGEINDY.ORG,2112293,92283.0,500749.0,1792933.0,990,201803199349308805
17,10878480,201806,UqU4ztC2HSE7wVYBZTGs5g,Executive Direc,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cross-Country for Youth,501(c)(3),N60,"Amateur Sports Clubs, Leagues, N.E.C.",6514 Chestnut Grove Lane,Charlotte,NC,28210,NaN,7.046345e+09,www.crosscountryforyouth.org,10890,437.0,54761.0,59565.0,990EZ,201901359349205060
22,10878619,201812,B1v49l2CYHxfzYv1hEotgw,President,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5475,NaN,NaN,5894.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Believe Ministries Inc,501(c)(3),X20,Christian,110 Roswell Farms Circle,Roswell,GA,300754351,NaN,7.705871e+09,http//www.believeministries.org/,67107,1662.0,45281.0,47202.0,990EZ,201920919349200217
24,10878619,201812,rIIk_772wAGfDvbFLpwNFw,Treasurer,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5475,NaN,NaN,5893.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Believe Ministries Inc,501(c)(3),X20,Christian,110 Roswell Farms Circle,Roswell,GA,300754351,NaN,7.705871e+09,http//www.believeministries.org/,67107,1662.0,45281.0,47202.0,990EZ,201920919349200217


Unsupervised learning - to identify clusters of similar NTEE descriptions, then based on these clusters, model to find salaries.
